In [0]:
%pip install --upgrade databricks-sdk --quiet
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.pipelines import *
from databricks.sdk.service.workspace import ImportFormat
import time

# Update the catalog name!
catalog_name = "<catalog_name>"

def run_mv_refresh_demo(catalog_name: str):
    w = WorkspaceClient()
    host = w.config.host
    user = w.current_user.me().user_name
    schema = "demo"
    table = f"{catalog_name}.{schema}.random_data"
    mv_path = f"/Users/{user}/mv_insights_query.sql"
    pipeline_name = f"{catalog_name}_mv_demo_pipeline_v2"

    def insert_data(with_random=False):
        print(f"\n👉 Inserting new data {'with' if with_random else 'without'} random_number column...")
        rand_col = ", CAST(rand() * 100 AS INT) AS random_number" if with_random else ""
        spark.sql(f"""
        INSERT INTO {table}
        SELECT 
            CAST(rand() * 100000 AS BIGINT) AS id,
            CASE CAST(rand() * 5 AS INT)
                WHEN 0 THEN 'Alice'
                WHEN 1 THEN 'Bob'
                WHEN 2 THEN 'Charlie'
                WHEN 3 THEN 'Diana'
                ELSE 'Eve'
            END AS name
            {rand_col}
        FROM RANGE(5)
        """)

    def upload_mv_sql(sql_text: str):
        print(f"\n📄 Uploading MV definition to: {mv_path}")
        w.workspace.upload(
            path=mv_path,
            content=sql_text.encode("utf-8"),
            format=ImportFormat.SOURCE,
            overwrite=True
        )

    def run_pipeline_and_wait():
        print(f"\n🚀 Starting pipeline update...")
        update = w.pipelines.start_update(pipeline.pipeline_id)
        update_id = update.update_id
        print(f"🔗 Update in progress: {host}/pipelines/{pipeline.pipeline_id}/updates/{update_id}")

        # Wait until update finishes
        state = None
        while True:
            current_update = w.pipelines.get_update(pipeline.pipeline_id, update_id)
            new_state = current_update.update.state
            if new_state != state:
                print(f"📌 Pipeline update state: {new_state}")
                state = new_state
            if new_state in [UpdateInfoState.COMPLETED, UpdateInfoState.FAILED, UpdateInfoState.CANCELED]:
                break
            time.sleep(5)

        print("✅ Pipeline update complete.")
        print("⏳ Waiting 10 seconds before continuing...\n")
        time.sleep(10)

    # Step 1–11: Existing logic
    print("\n🗑️ Drop schema and initial table if previously created...")
    spark.sql(f"DROP SCHEMA IF EXISTS {catalog_name}.{schema} CASCADE")

    print("\n📁 Creating catalog, schema, and initial table...")
    spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema}")
    spark.sql(f"""
    CREATE OR REPLACE TABLE {table} (
        id STRING,
        name STRING
    )
    USING DELTA
    TBLPROPERTIES (
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true',
        'delta.enableRowTracking' = 'true',
        'delta.enableDeletionVectors' = 'true',
        'delta.feature.allowColumnDefaults' = 'supported'
    )
    """)
    insert_data(with_random=False)

    print("\n🧱 Creating initial materialized view (no random)...")
    upload_mv_sql(f"""
    CREATE MATERIALIZED VIEW random_data_MV_v2
    AS
    SELECT
        name,
        COUNT(id) AS name_count
    FROM {table}
    GROUP BY ALL
    """)

    print("\n🔧 Creating pipeline...")
    existing = w.pipelines.list_pipelines(filter=f"name LIKE '%{pipeline_name}%'")
    for p in existing:
        print(f"🗑️ Deleting existing pipeline: {p.pipeline_id}")
        w.pipelines.delete(p.pipeline_id)

    pipeline = w.pipelines.create(
        catalog=catalog_name,
        continuous=False,
        channel="PREVIEW",
        name=pipeline_name,
        schema=schema,
        libraries=[
            PipelineLibrary(glob=PathPattern(include=mv_path))
        ],
        serverless=True,
        event_log=EventLogSpec(catalog=catalog_name, schema=schema, name="event_log_incremental_refresh_demo_v2")
    )

    print(f"✅ Created pipeline ID: {pipeline.pipeline_id}")
    print(f"🔗 Pipeline link: {host}/pipelines/{pipeline.pipeline_id}")

    run_pipeline_and_wait()  # Step 4

    insert_data()            # Step 5
    run_pipeline_and_wait()

    print("\n🔁 Updating MV to use RANDOM() in the SELECT...")  # Step 6
    upload_mv_sql(f"""
    CREATE MATERIALIZED VIEW random_data_MV_v2
    AS
    SELECT
        name,
        RANDOM() as random_number,
        COUNT(id) AS name_count
    FROM {table}
    GROUP BY ALL
    """)

    insert_data()            # Step 7
    run_pipeline_and_wait()

    print("\n📐 Altering table to add 'random_number' column...")  # Step 8
    spark.sql(f"ALTER TABLE {table} ADD COLUMNS (random_number INT)")

    insert_data(with_random=True)  # Step 9

    print("\n🔁 Updating MV to pull 'random_number' from source instead of RANDOM()...")  # Step 10
    upload_mv_sql(f"""
    CREATE MATERIALIZED VIEW random_data_MV_v2
    AS
    SELECT
        name,
        random_number,
        COUNT(id) AS name_count
    FROM {table}
    GROUP BY ALL
    """)
    run_pipeline_and_wait()

    insert_data(with_random=True)  # Step 11
    run_pipeline_and_wait()

    print("🏁 Demo complete.")

run_mv_refresh_demo(catalog_name)


🗑️ Drop schema and initial table if previously created...

📁 Creating catalog, schema, and initial table...

👉 Inserting new data without random_number column...

🧱 Creating initial materialized view (no random)...

📄 Uploading MV definition to: /Users/andrea.tardif@databricks.com/mv_insights_query.sql

🔧 Creating pipeline...
🗑️ Deleting existing pipeline: a47df3f7-d267-4783-a9c1-d5d7726683a6
✅ Created pipeline ID: e6e8285f-7f7f-4489-9f31-d2e2105d89e3
🔗 Pipeline link: https://e2-demo-field-eng.cloud.databricks.com/pipelines/e6e8285f-7f7f-4489-9f31-d2e2105d89e3

🚀 Starting pipeline update...
🔗 Update in progress: https://e2-demo-field-eng.cloud.databricks.com/pipelines/e6e8285f-7f7f-4489-9f31-d2e2105d89e3/updates/18bbad4b-b60f-485d-8527-635580e95cc1
📌 Pipeline update state: UpdateInfoState.CREATED
📌 Pipeline update state: UpdateInfoState.WAITING_FOR_RESOURCES
📌 Pipeline update state: UpdateInfoState.INITIALIZING
📌 Pipeline update state: UpdateInfoState.SETTING_UP_TABLES
📌 Pipeline upda

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Query the event log
event_log = spark.sql(f"""
    SELECT timestamp, message, details
    FROM {catalog_name}.demo.event_log_incremental_refresh_demo_v2
    WHERE event_type = 'planning_information'
    ORDER BY timestamp DESC
""")

# Add row number
window_spec = Window.orderBy(event_log["timestamp"].asc())
event_log_with_rownum = event_log.withColumn("row_num", row_number().over(window_spec))

# Reorder columns if you'd like
event_log_with_rownum = event_log_with_rownum.select("row_num", "message", "details")

# Display
event_log_with_rownum.display()


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1056: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


row_num,message,details
1,Flow 'andrea_tardif.demo.random_data_mv_v2' has been planned in DLT to be executed as COMPLETE_RECOMPUTE.,"{""planning_information"":{""technique_information"":[{""incrementalization_issues"":[{""issue_type"":""CHANGE_SET_MISSING"",""prevent_incrementalization"":true}]},{""maintenance_type"":""MAINTENANCE_TYPE_COMPLETE_RECOMPUTE"",""is_chosen"":true,""is_applicable"":true,""cost"":804.0}],""source_table_information"":[{""table_name"":""`andrea_tardif`.`demo`.`random_data`"",""table_id"":""fc63d123-c018-43f8-af10-b8d7e68aec1b"",""catalog_table_type"":""MANAGED"",""full_size"":878.0,""num_rows"":5,""num_files"":1,""is_size_after_pruning"":true,""is_row_id_enabled"":true,""is_cdf_enabled"":false,""is_deletion_vector_enabled"":true}],""target_table_information"":{""table_name"":""`andrea_tardif`.`demo`.`random_data_mv_v2`"",""table_id"":""2f166c3b-9765-4280-b01b-6669b814e54f"",""full_size"":0.0,""is_row_id_enabled"":true,""is_cdf_enabled"":true,""is_deletion_vector_enabled"":true},""planning_wall_time_ms"":2090}}"
2,Flow 'andrea_tardif.demo.random_data_mv_v2' has been planned in DLT to be executed as GROUP_AGGREGATE.,"{""planning_information"":{""technique_information"":[{""maintenance_type"":""MAINTENANCE_TYPE_NO_OP"",""incrementalization_issues"":[{""issue_type"":""DATA_HAS_CHANGED"",""prevent_incrementalization"":true}]},{""maintenance_type"":""MAINTENANCE_TYPE_COMPLETE_RECOMPUTE"",""is_chosen"":false,""is_applicable"":true,""cost"":1616.0},{""maintenance_type"":""MAINTENANCE_TYPE_GROUP_AGGREGATE"",""is_chosen"":true,""is_applicable"":true}],""source_table_information"":[{""table_name"":""`andrea_tardif`.`demo`.`random_data`"",""table_id"":""fc63d123-c018-43f8-af10-b8d7e68aec1b"",""catalog_table_type"":""MANAGED"",""full_size"":1763.0,""num_rows"":10,""num_files"":2,""change_size"":885.0,""num_changed_rows"":5,""num_rows_in_changed_files"":5,""num_changed_files"":1,""change_file_read_size"":885.0,""is_size_after_pruning"":true,""is_row_id_enabled"":true,""is_cdf_enabled"":false,""is_deletion_vector_enabled"":true,""is_change_from_legacy_cdf"":false}],""target_table_information"":{""table_name"":""`andrea_tardif`.`demo`.`random_data_mv_v2`"",""table_id"":""2f166c3b-9765-4280-b01b-6669b814e54f"",""full_size"":1226.0,""is_row_id_enabled"":true,""is_cdf_enabled"":true,""is_deletion_vector_enabled"":true},""planning_wall_time_ms"":8728}}"
3,Flow 'andrea_tardif.demo.random_data_mv_v2' has been planned in DLT to be executed as COMPLETE_RECOMPUTE.,"{""planning_information"":{""technique_information"":[{""maintenance_type"":""MAINTENANCE_TYPE_NO_OP"",""incrementalization_issues"":[{""issue_type"":""DATA_HAS_CHANGED"",""prevent_incrementalization"":true}]},{""incrementalization_issues"":[{""issue_type"":""PLAN_NOT_DETERMINISTIC"",""prevent_incrementalization"":true,""expression_name"":""Rand"",""plan_not_deterministic_sub_type"":""NON_DETERMINISTIC_EXPRESSION""},{""issue_type"":""QUERY_FINGERPRINT_CHANGED"",""prevent_incrementalization"":true,""fingerprint_diff_before"":"": {\\n \\\""product-class\\\"" : \\\""ExprId\\\"",\\n \\\""id\\\"" : 1\\n },\\n \\\""qualifier\\\"" : [ ]\\n } ]^^^ ],\\n \\\""aggregateExpressions\\\"" : [ [ {\\n \\\""class\\\"" : \\\""AttributeReference\\\"",\\n \\\""num-children\\\"" : 0,\\n \\\""name\\\"" : \\\""none\\\"",\\n \\\""dataType\\\"" : \\\""string\\\"",\\n \\\""nullable\\\"" : true,\\n \\\""metadata\\\"" : { },\\n \\\""exprId\\\"" : {\\n \\\""product-class\\\"" : \\\""ExprId\\\"",\\n \\\""id\\\"" : 1\\n },\\n \\\""qualifier\\\"" : [ ]\\n } ], [ {\\n \\\""class\\\"" : \\\""Alias\\\"",\\n \\\""num-children\\\"" : 1,\\n \\\""child\\\"" : 0,\\n \\\""name\\\"" : \\\""\\\"",\\n \\\""exprId\\\"" : {\\n \\\""product-class\\\"" : \\\""ExprId\\\"",\\n \\\""id\\\"" : 0\\n },\\n \\\""qualifier\\\"" : [ ],\\n \\\""nonInheritableMetadataKeys\\\"" : [ ]\\n }, {\\n \\\""class\\\"" : \\\""aggregate.AggregateExpression\\\"",\\n \\\""num-children\\\"" : 1,\\n \\\""aggregateFunction\\\"" : 0,\\n \\